<a href="https://colab.research.google.com/github/bngaruiya/CropDiseaseClassifier/blob/master/crop_diseases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Crop Diseases/"

In [0]:
%cd /content/gdrive/My Drive/Crop Diseases

/content/gdrive/My Drive/Crop Diseases


In [0]:
# !kaggle datasets download -d emmarex/plantdisease

In [0]:
# !unzip \*.zip  && rm *.zip

In [0]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import time
import torchvision
import torch.nn.functional as F
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch
from torch.optim.lr_scheduler import StepLR
from collections import OrderedDict
from PIL import Image
import json

In [0]:
train_on_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
data_dir = 'PlantVillage'
train_dir = os.path.join(data_dir, 'train/')
val_dir = os.path.join(data_dir, 'val/')
test_dir = os.path.join(data_dir, 'test/')

In [0]:
train_transforms = transforms.Compose([transforms.Resize(size=224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(10),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.4548, 0.4758, 0.3215],
                                                          [0.4548, 0.4758, 0.3215])])
valid_transforms = transforms.Compose([transforms.Resize(size=224),
                                        transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.4548, 0.4758, 0.3215],
                                                          [0.4548, 0.4758, 0.3215])])

test_transforms = transforms.Compose([transforms.Resize(size=224),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.4548, 0.4758, 0.3215],
                                                          [0.4548, 0.4758, 0.3215])])

In [0]:
train_dataset = datasets.ImageFolder(
    train_dir,
    transform = train_transforms
)
val_dataset = datasets.ImageFolder(
    val_dir,
    transform = valid_transforms
)
test_dataset = datasets.ImageFolder(
    test_dir,
    transform = test_transforms
)

In [0]:
batch_size = 128

In [0]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [0]:
classes = os.listdir(train_dir)
print(classes)

['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato__Target_Spot', 'Tomato__Tomato_mosaic_virus', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato_healthy']


In [0]:
train_dataset.class_to_idx
idx_to_class = {j:i for i, j in train_dataset.class_to_idx.items()}

with open('classes.json', 'w') as f:
    json.dump(idx_to_class, f)

In [0]:
model_transfer = models.resnet152(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model_transfer.parameters():
    param.requires_grad = False

# Creating the classifier ordered dictionary first
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(2048, 512)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(512, (len(classes)-1))),
                          ]))

# Replacing the pretrained model classifier with our classifier
model_transfer.fc = classifier

if train_on_gpu:
    model_transfer.cuda()

#Let's check the model architecture:
print(model_transfer)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/checkpoints/resnet152-b121ed2d.pth



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1,

In [0]:
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.Adam(model_transfer.fc.parameters(), lr = 0.0001)
# scheduler_transfer = StepLR(optimizer_transfer, step_size=5, gamma=0.2)

In [0]:
# train
def train (epochs, loaders, model, optimizer, criterion, train_on_gpu, save_path):
    valid_loss_min = np.Inf

    # scheduler = scheduler
    for epoch in range(1, epochs+1):
        # keep track of the training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        #####################
        ## train the model ##
        #####################
        
        for batch_i, (data, target) in enumerate(loaders['train']):
            ## Move data and target to GPU
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            ## Zero the gradients
            optimizer.zero_grad()
            ## Forward pass
            output = model(data)
            ## Calculate the loss
            loss = criterion(output, target)
            ## backpropagation
            loss.backward()
            ## Single step optimization
            optimizer.step()
            ## Update the training loss
            train_loss += ((1 / (batch_i + 1)) * (loss.data - train_loss))

            if batch_i % 5 == 4 : #Print training loss for every 32 batches
                print('Epoch %d, Batch %d, loss %.16f' %(epoch, batch_i + 1, train_loss / 5))
                train_loss = 0.0

        ######################
        # Validate the model #
        ######################

        # model_eval()
        for batch_i, (data, target) in enumerate(loaders['valid']):
            # move data and target to GPU
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            with torch.no_grad():
                output = model(data)
            loss = criterion(output, target)
            valid_loss += ((1 / (batch_i + 1)) * (loss.data - valid_loss))
            
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_loss_min, valid_loss))
            torch.save(model.state_dict(), save_path)
            # torch.save(model, save_path)
            valid_loss_min = valid_loss
        # scheduler.step()
            
    return model

In [0]:
## Define the loaders
loaders_transfer = {
    "train": train_loader,
    "valid": val_loader,
    "testing": test_loader
}

In [0]:
model_transfer = train(5, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, train_on_gpu, 'model_transfer.pt')

Epoch 1, Batch 5, loss 0.3241525590419769
Epoch 1, Batch 10, loss 0.2402738630771637
Epoch 1, Batch 15, loss 0.2371434718370438
Epoch 1, Batch 20, loss 0.1890543550252914
Epoch 1, Batch 25, loss 0.1647123098373413
Epoch 1, Batch 30, loss 0.1206478402018547
Epoch 1, Batch 35, loss 0.0992157161235809
Epoch 1, Batch 40, loss 0.0702532529830933
Epoch 1, Batch 45, loss 0.0571819730103016
Epoch 1, Batch 50, loss 0.0597537867724895
Epoch 1, Batch 55, loss 0.0562696233391762
Epoch 1, Batch 60, loss 0.0458182059228420
Epoch 1, Batch 65, loss 0.0488610751926899
Epoch 1, Batch 70, loss 0.0482697784900665
Epoch 1, Batch 75, loss 0.0433117933571339
Epoch 1, Batch 80, loss 0.0377945266664028
Epoch 1, Batch 85, loss 0.0414485111832619
Epoch 1, Batch 90, loss 0.0356950722634792
Epoch 1, Batch 95, loss 0.0302480701357126
Epoch 1, Batch 100, loss 0.0289309304207563
Epoch 1, Batch 105, loss 0.0290929879993200
Epoch 1, Batch 110, loss 0.0262486319988966
Epoch 1, Batch 115, loss 0.0238273628056049
Epoch 1,

In [0]:
!mkdir -p '/content/gdrive/My Drive/Crop Diseases/models/'
!cp model_transfer.pt '/content/gdrive/My Drive/Crop Diseases/models/'

In [0]:
model_transfer.load_state_dict(torch.load('model_transfer.pt'), ) 

<All keys matched successfully>

In [0]:
def test(loaders, model, criterion, use_cuda):
    # Monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
    
    # model_eval()
    for batch_i, (data, target) in enumerate(loaders['testing']):
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        loss = criterion(output, target)
        test_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))
    
test(loaders_transfer, model_transfer, criterion_transfer, train_on_gpu)

Test Loss: 62.154259


Test Accuracy:  6% (212/3190)


In [0]:
def predict_crop_disease(img_path):
    # load the image and return the predicted breed
    img = Image.open(img_path).convert('RGB')
    transform = transforms.Compose([transforms.Resize(size=224),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.4548, 0.4758, 0.3215],
                                                          [0.4548, 0.4758, 0.3215])])
    img_tensor = transform(img).unsqueeze(0)
    if train_on_gpu:
        img_tensor = img_tensor.cuda()
    # Turn on evaluation mode
    model_transfer.eval()
    # Get the predicted class
    with torch.no_grad():
        output = model_transfer(img_tensor)
        pred = torch.argmax(output).item()
        # target = torch.max(target.data)

    #Turn off evaluation mode
    # model_transfer.train()
    # Get the predicted class using the defined model
    crop_disease_class = classes[pred]
    return crop_disease_class

In [0]:
def run(path):
    for img in (path):
        img = Image.open(path)
        plt.show(img)
        plt.show()
        print(f'The crop disease in this photo is ... {predict_crop_disease(path)}')
        print('################################################')

In [0]:
test_dir = '/content/gdrive/My Drive/Crop Diseases/predict/'
for file in os.listdir(test_dir):
    run(test_dir + file)

UnboundLocalError: ignored